In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import jieba
import pprint

from os import path
from PIL import Image
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## 基于Jieba的三种分词示例

- jieba.cut 方法接受三个输入参数: 需要分词的字符串；cut_all 参数用来控制是否采用全模式；HMM 参数用来控制是否使用HMM 模型
- jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。 
  <br/>注意：不建议直接输入 GBK 字符串，可能无法预料地错误解码成 UTF-8
- jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)，或者用 `jieba.lcut` 以及 `jieba.lcut_for_search` 直接返回 list
- jieba.Tokenizer(dictionary=DEFAULT_DICT) 新建自定义分词器，可用于同时使用不同词典。jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。


In [3]:
#coding=utf-8
import jieba,math
import jieba.analyse

#jieba.cut主要有三种模式
#随便对一个动物园的评论进行分析
str_text="真是好久好久没来哈皮娜拉野生动物园了，记忆里还是小时候三四年级学校组织春游去的银河系。jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。"
#全模式cut_all=True
cut1 = jieba.cut(str_text,cut_all=True)
print(type(cut1))
lst_quan1 = list(cut1)
print('全模式分词：{ %d}' % len(lst_quan1))
print("/".join(lst_quan1))


Building prefix dict from the default dictionary ...


<class 'generator'>


Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.633 seconds.
Prefix dict has been built succesfully.


全模式分词：{ 106}
真是/好久/好久好久/好久/好久没/没来/哈/皮/娜拉/野生/野生动物/生动/动物/动物园/了///记忆/记忆里/还是/小时/小时候/时候/三四/三四年/四年/四年级/年级/学校/组织/春游/游去/的/银河/银河系/河系//jieba/cut/for/search//方法/接受/两个/参数///需要/分词/的/字符/字符串///是否/使用//HMM//模型///该/方法/适合/合用/用于/搜索/搜索引擎/索引/引擎/构建/倒排/索引/的/分词///粒度/比较/细/待/分词/的/字符/字符串/可以/以是//unicode//或//UTF/8//字符/字符串//GBK//字符/字符串//


In [4]:
#精准模式cut_all=False，默认即是
cut2=jieba.cut(str_text,cut_all=False)
lst_jing2 = list(cut2)
print('精准模式分词：{ %d}' % len(lst_jing2))
print("/".join(lst_jing2))


精准模式分词：{ 80}
真是/好久好久/没来/哈皮/娜拉/野生/动物园/了/，/记忆里/还是/小时候/三四年/级/学校/组织/春游/去/的/银河系/。/jieba/./cut/_/for/_/search/ /方法/接受/两个/参数/：/需要/分词/的/字符串/；/是否/使用/ /HMM/ /模型/。/该/方法/适合/用于/搜索引擎/构建/倒排/索引/的/分词/，/粒度/比较/细待/分词/的/字符串/可以/是/ /unicode/ /或/ /UTF/-/8/ /字符串/、/GBK/ /字符串/。


In [7]:


#搜索引擎模式  cut_for_search
cut_search=jieba.cut_for_search(str_text)
lst_search = list(cut_search)
print('搜索引擎分词：{ %d}' % len(lst_search))
print("/".join(lst_search))

搜索引擎分词：{ 97}
真是/好久/好久/好久好久/没来/哈皮/娜拉/野生/动物/动物园/了/，/记忆/记忆里/还是/小时/时候/小时候/三四/四年/三四年/级/学校/组织/春游/去/的/银河/河系/银河系/。/jieba/./cut/_/for/_/search/ /方法/接受/两个/参数/：/需要/分词/的/字符/字符串/；/是否/使用/ /HMM/ /模型/。/该/方法/适合/用于/搜索/索引/引擎/搜索引擎/构建/倒排/索引/的/分词/，/粒度/比较/细待/分词/的/字符/字符串/可以/是/ /unicode/ /或/ /UTF/-/8/ /字符/字符串/、/GBK/ /字符/字符串/。


## 关键词提取、关键词提取

In [15]:
import jieba.analyse
#analyse.extract.tags

print(str_text)
keywords1=jieba.analyse.extract_tags(str_text)
print('关键词提取:'+"/".join(keywords1))

keywords_top=jieba.analyse.extract_tags(str_text,topK=3)
print('关键词topk:'+"/".join(keywords_top))     #有时不确定提取多少关键词，可利用总词的百分比
print('总词数:{}'.format(len(list(jieba.cut(str_text)))))
total=len(list(jieba.cut(str_text)))
get_cnt=math.ceil(total*0.1)  #向上取整
print('从%d 中取出%d 个词'% (total,get_cnt))
keywords_top1=jieba.analyse.extract_tags(str_text,topK=get_cnt)
print('关键词topk'+"/".join(keywords_top1))


真是好久好久没来哈皮娜拉野生动物园了，记忆里还是小时候三四年级学校组织春游去的银河系。jieba.cut_for_search 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。
关键词提取:分词/字符串/哈皮/jieba/cut/search/HMM/倒排/细待/unicode/UTF/GBK/春游/粒度/好久好久/方法/记忆里/三四年/搜索引擎/娜拉
关键词topk:分词/字符串/哈皮
总词数:80
从80 中取出8 个词
关键词topk分词/字符串/哈皮/jieba/cut/search/HMM/倒排


## 添加自定义词

处理时，add_word(word,freq=None,tag=None)和del_word可在程序中动态修改词典。  
suggest_freq(segment,tune=Ture)可调节单词词频，时期能或不能显示  
`注：自动计算的词频在使用HMM新词发现功能时可能无效`


In [25]:
str_jing2=jieba.cut(str_text,cut_all=False)
print('add_word前:'+"/".join(str_jing2))
#添加自定义词
jieba.del_word('哈皮娜拉')
jieba.del_word('野生')
str_jing3=jieba.cut(str_text,cut_all=False)
print('add_word后:'+"/".join(str_jing3))
#修正词频
jieba.suggest_freq('野生动物园',tune=True)
str_jing4=jieba.cut(str_text,cut_all=False)
print('suggest_freq后:'+"/".join(str_jing4))


add_word前:真是/好久好久/没来/哈皮/娜拉/野生动物园/了/，/记忆里/还是/小时候/三四年/级/学校/组织/春游/去/的/银河系/。/jieba/./cut/_/for/_/search/ /方法/接受/两个/参数/：/需要/分词/的/字符串/；/是否/使用/ /HMM/ /模型/。/该/方法/适合/用于/搜索引擎/构建/倒排/索引/的/分词/，/粒度/比较/细待/分词/的/字符串/可以/是/ /unicode/ /或/ /UTF/-/8/ /字符串/、/GBK/ /字符串/。
add_word后:真是/好久好久/没来/哈皮/娜拉/野生动物园/了/，/记忆里/还是/小时候/三四年/级/学校/组织/春游/去/的/银河系/。/jieba/./cut/_/for/_/search/ /方法/接受/两个/参数/：/需要/分词/的/字符串/；/是否/使用/ /HMM/ /模型/。/该/方法/适合/用于/搜索引擎/构建/倒排/索引/的/分词/，/粒度/比较/细待/分词/的/字符串/可以/是/ /unicode/ /或/ /UTF/-/8/ /字符串/、/GBK/ /字符串/。
suggest_freq后:真是/好久好久/没来/哈皮/娜拉/野生动物园/了/，/记忆里/还是/小时候/三四年/级/学校/组织/春游/去/的/银河系/。/jieba/./cut/_/for/_/search/ /方法/接受/两个/参数/：/需要/分词/的/字符串/；/是否/使用/ /HMM/ /模型/。/该/方法/适合/用于/搜索引擎/构建/倒排/索引/的/分词/，/粒度/比较/细待/分词/的/字符串/可以/是/ /unicode/ /或/ /UTF/-/8/ /字符串/、/GBK/ /字符串/。


## 加载自定义词库

jieba.load_userdict(filename)  
#filename为文件路径词典格式和dict.txt一样，一词一行，每行分三个部分（用空格隔开），词语 词频（可省） 词性（可省）顺序不可颠倒，若filename为路径或二进制方式打开，则需为UTF-8
'''
#定义：三四年级  在文件内
jieba.load_userdict('C:\\Users\\lenovo\\Desktop\\自定义词库.txt')
str_load=jieba.cut(str_text,cut_all=False)
print('load_userdict后:'+"/".join(str_load))
'''
注jieba.load_userdict加载自定义词库和jieba初始化词库一同使用，但是，默认的初始词库放在安装目录ixia，如果确定长期加载词库，就替换他使用词库的切换功能set_dictionary()
可将jieba默认词库copy到自己的目录下，在添加，或者找到更全的词库  
'''
#一般在python都为site-packages\jieba\dict.txt  
#模拟演示  
jieba.set_dictionary('filename')  
#之后进行分词，如果我们切换了词库，此时程序就会初始化我们制定的词库，而不加载默认路径词库  
